<a href="https://colab.research.google.com/github/KanmaniKannathasan/Energy_Rating/blob/main/KK_DataMining_BER_Rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the relevant libraries
import numpy as np
import pandas as pd
from pandas import read_csv, get_dummies,DataFrame,Series
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from plotly import graph_objs, figure_factory
import joblib
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#setting the option to ensure we can see all the columns in output window
pd.set_option('display.max_columns', None)

#viewing the first 5 rows of the data ;
dataset.head()

,CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,UValueFloor,UValueWindow,UvalueDoor,WallArea,RoofArea,FloorArea,WindowArea,DoorArea,NoStoreys,CO2Rating,MainSpaceHeatingFuel,MainWaterHeatingFuel,HSMainSystemEfficiency,MultiDwellingMPRN,TGDLEdition,MPCDERValue,HSEffAdjFactor,HSSupplHeatFraction,HSSupplSystemEff,WHMainSystemEff,WHEffAdjFactor,SupplSHFuel,SupplWHFuel,SHRenewableResources,WHRenewableResources,NoOfChimneys,NoOfOpenFlues,NoOfFansAndVents,NoOfFluelessGasFires,DraftLobby,VentilationMethod,FanPowerManuDeclaredValue,HeatExchangerEff,StructureType,SuspendedWoodenFloor,PercentageDraughtStripped,NoOfSidesSheltered,PermeabilityTest,PermeabilityTestResult,TempAdjustment,HeatSystemControlCat,HeatSystemResponseCat,NoCentralHeatingPumps,CHBoilerThermostatControlled,NoOilBoilerHeatingPumps,OBBoilerThermostatControlled,OBPumpInsideDwelling,NoGasBoilerHeatingPumps,WarmAirHeatingSystem,UndergroundHeating,GroundFloorUValue,DistributionLosses,StorageLosses,ManuLossFactorAvail,SolarHotWaterHeating,ElecImmersionInSummer,CombiBoiler,KeepHotFacility,WaterStorageVolume,DeclaredLossFactor,TempFactorUnadj,TempFactorMultiplier,InsulationType,InsulationThickness,PrimaryCircuitLoss,CombiBoilerAddLoss,ElecConsumpKeepHot,ApertureArea,ZeroLossCollectorEff,CollectorHeatLossCoEff,AnnualSolarRadiation,OvershadingFactor,CylinderStat,SolarStorageVolume,VolumeOfPreHeatStore,CombinedCylinder,ElectricityConsumption,SWHPumpSolarPowered,ChargingBasisHeatConsumed,gsdHSSupplHeatFraction,gsdHSSupplSystemEff,DistLossFactor,CHPUnitHeatFraction,CHPSystemType,CHPElecEff,CHPHeatEff,CHPFuelType,SupplHSFuelTypeID,gsdSHRenewableResources,gsdWHRenewableResources,SolarHeatFraction,DeliveredLightingEnergy,DeliveredEnergyPumpsFans,DeliveredEnergyMainWater,DeliveredEnergyMainSpace,PrimaryEnergyLighting,PrimaryEnergyPumpsFans,PrimaryEnergyMainWater,PrimaryEnergyMainSpace,CO2Lighting,CO2PumpsFans,CO2MainWater,CO2MainSpace,GroundFloorArea,GroundFloorHeight,FirstFloorArea,FirstFloorHeight,SecondFloorArea,SecondFloorHeight,ThirdFloorArea,ThirdFloorHeight,ThermalBridgingFactor,ThermalMassCategory,PredominantRoofTypeArea,PredominantRoofType,LowEnergyLightingPercent,TotalDeliveredEnergy,DeliveredEnergySecondarySpace,DeliveredEnergySupplementaryWater,LivingAreaPercent,CO2SecondarySpace,CO2SupplementaryWater,PrimaryEnergySecondarySpace,PrimaryEnergySupplementaryWater,HESSchemeUpgrade,RoomInRoofArea,PurposeOfRating,DateOfAssessment,FirstEnergyTypeId,FirstEnergyType_Description,FirstEnerProdComment,FirstEnerProdDelivered,FirstPartLTotalContribution,FirstEnerProdConvFactor,FirstEnerProdCO2EmissionFactor,FirstEnerConsumedComment,FirstEnerConsumedDelivered,FirstEnerConsumedConvFactor,FirstEnerConsumedCO2EmissionFactor,SecondEnergyTypeId,SecondEnergyType_Description,SecondEnerProdComment,SecondEnerProdDelivered,SecondPartLTotalContribution,SecondEnerProdConvFactor,SecondEnerProdCO2EmissionFactor,SecondEnerConsumedComment,SecondEnerConsumedDelivered,SecondEnerConsumedConvFactor,SecondEnerConsumedCO2EmissionFactor,ThirdEnergyTypeId,ThirdEnergyType_Description,ThirdEnerProdComment,ThirdEnerProdDelivered,ThirdPartLTotalContribution,ThirdEnerProdConvFactor,ThirdEnerProdCO2EmissionFactor,ThirdEnerConsumedComment,ThirdEnerConsumedDelivered,ThirdEnerConsumedConvFactor,ThirdEnerConsumedCO2EmissionFactor,FirstBoilerFuelType,FirstHeatGenPlantEff,FirstPercentageHeat,SecondBoilerFuelType,SecondHeatGenPlantEff,SecondPercentageHeat,ThirdBoilerFuelType,ThirdHeatGenPlantEff,ThirdPercentageHeat,SolarSpaceHeatingSystem,TotalPrimaryEnergyFact,TotalCO2Emissions,FirstWallType_Description,FirstWallDescription,FirstWallArea,FirstWallUValue,FirstWallIsSemiExposed,FirstWallAgeBandId,FirstWallTypeId,SecondWallType_Description,SecondWallDescription,SecondWallArea,SecondWallUValue,SecondWallIsSemiExposed,SecondWallAgeBandId,SecondWallTypeId,ThirdWallType_Description,ThirdWallDescription,ThirdWallArea,ThirdWallUValue,ThirdWallIsSemiExpo

In [ ]:
#viewing information about the data and the data type of each variable
dataset.info(verbose=True, null_counts=True)

In [ ]:
#dataset['NoOfFluelessGasFires'].value_counts() #viewing the unique values in a column

# **Data Preparation**

In [ ]:
# Dropping unnecessary columns (part 1). Columns with index 135 to 210
dataset.drop(dataset.iloc[:, 135:211], axis = 1, inplace = True)

# view information about the modified dataset
dataset.info(verbose=True, null_counts=True)

In [ ]:
# Dropping unnecessary columns (part 2)
dataset.drop(dataset.columns[[0,1,2,3,5,19,20,23,24,25,26,29,32,33,42,46,58,59,62,66,67,68,69,70,71,72,74,75,76,77,78,79,80,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,122,123,125,126,127,129,130,131,132,133]], axis=1, inplace=True)

# view information about the modified dataset
dataset.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 53 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   EnergyRating                  200000 non-null  object 
 1   GroundFloorArea(sq m)         200000 non-null  float64
 2   UValueWall                    200000 non-null  float64
 3   UValueRoof                    200000 non-null  float64
 4   UValueFloor                   200000 non-null  float64
 5   UValueWindow                  200000 non-null  float64
 6   UvalueDoor                    200000 non-null  float64
 7   WallArea                      200000 non-null  float64
 8   RoofArea                      200000 non-null  float64
 9   FloorArea                     200000 non-null  float64
 10  WindowArea                    200000 non-null  float64
 11  DoorArea                      200000 non-null  float64
 12  NoStoreys                     200000 non-nul

In [ ]:
#After removal of irrelevant variables/columns we still have 13 columns with some null values. We will examine the records to get more insight into the null values

pd.set_option("display.max_rows", 200) #setting the option to ensure we can see all the rows in output window
dataset_with_null = dataset.loc[dataset.isnull().any(axis=1)] #creating a data subset with only records that have null values in any column
dataset_with_null.head(60) #viewing the top 60 rows of the data subset

In [ ]:
#dropping all rows that have any null values
dataset_no_null = dataset.dropna()

# view information about the modified dataset
dataset_no_null.info(verbose=True, null_counts=True)

In [ ]:
#viewing a summary description of the categorical variables
dataset_no_null.describe(include = ['object']).T

,count,unique,top,freq
EnergyRating,195312,15,D1,26275
MultiDwellingMPRN,195312,2,NO,193891
DraftLobby,195312,2,NO,156601
VentilationMethod,195312,6,Natural vent.,192809
SuspendedWoodenFloor,195312,3,No,177214
CHBoilerThermostatControlled,195312,2,NO,131910
OBBoilerThermostatControlled,195312,2,NO,176485
OBPumpInsideDwelling,195312,2,NO,174838
WarmAirHeatingSystem,195312,2,NO,194582
StorageLosses,195312,2,YES,187084


In [ ]:
#viewing the list of unique values in each categorical variable
print(dataset_no_null['EnergyRating'].unique())
print(dataset_no_null['MultiDwellingMPRN'].unique())
print(dataset_no_null['DraftLobby'].unique())
print(dataset_no_null['VentilationMethod'].unique())
print(dataset_no_null['SuspendedWoodenFloor'].unique())
print(dataset_no_null['CHBoilerThermostatControlled'].unique())
print(dataset_no_null['OBBoilerThermostatControlled'].unique())
print(dataset_no_null['OBPumpInsideDwelling'].unique())
print(dataset_no_null['WarmAirHeatingSystem'].unique())
print(dataset_no_null['StorageLosses'].unique())
print(dataset_no_null['SolarHotWaterHeating'].unique())
print(dataset_no_null['ElecImmersionInSummer'].unique())
print(dataset_no_null['CombiBoiler'].unique())
print(dataset_no_null['PrimaryCircuitLoss'].unique())
print(dataset_no_null['CylinderStat'].unique())
print(dataset_no_null['ThermalMassCategory'].unique())

['C2' 'D1' 'G ' 'B3' 'C3' 'D2' 'E2' 'F ' 'C1' 'E1' 'B2' 'B1' 'A3' 'A1'
 'A2']
['NO' 'YES']
['NO' 'YES']
['Natural vent.' 'Bal.whole mech.vent heat recvr'
 'Whole house extract vent.' 'Pos input vent.- outside'
 'Bal.whole mech.vent no heat re' 'Pos input vent.- loft']
['No                            ' 'Yes (Unsealed)                '
 'Yes (Sealed)                  ']
['NO ' 'YES']
['NO ' 'YES']
['NO ' 'YES']
['NO ' 'YES']
['YES' 'NO ']
['NO ' 'YES']
['NO ' 'YES']
['None                          ' 'Instantaneous, without keep-ho'
 'Instantaneous, with keep-hot f' 'Storage combi boiler store vol']
['Boiler with uninsulated primar' 'Electric immersion heater     '
 'Boiler with insulated primary ' 'Combi boiler                  '
 'None                          ' 'Separate boiler and thermal st'
 'CPSU (including electric CPSU)' 'Boiler and thermal store withi'
 'Community heating             ']
['NO ' 'YES']
['Medium              ' 'Medium-high         ' 'High                '
 'Low    

In [ ]:
#Encoding the categorical variables that have only 2 unique values
#Using Mapping
dataset_no_null['MultiDwellingMPRN'] = dataset_no_null['MultiDwellingMPRN'].map({'YES':1, 'NO':0})
dataset_no_null['DraftLobby'] = dataset_no_null['DraftLobby'].map({'YES':1, 'NO':0})
dataset_no_null['CHBoilerThermostatControlled'] = dataset_no_null['CHBoilerThermostatControlled'].map({'YES':1, 'NO ':0})
dataset_no_null['OBBoilerThermostatControlled'] = dataset_no_null['OBBoilerThermostatControlled'].map({'YES':1, 'NO ':0})
dataset_no_null['OBPumpInsideDwelling'] = dataset_no_null['OBPumpInsideDwelling'].map({'YES':1, 'NO ':0})
dataset_no_null['WarmAirHeatingSystem'] = dataset_no_null['WarmAirHeatingSystem'].map({'YES':1, 'NO ':0})
dataset_no_null['StorageLosses'] = dataset_no_null['StorageLosses'].map({'YES':1, 'NO ':0})
dataset_no_null['SolarHotWaterHeating'] = dataset_no_null['SolarHotWaterHeating'].map({'YES':1, 'NO ':0})
dataset_no_null['ElecImmersionInSummer'] = dataset_no_null['ElecImmersionInSummer'].map({'YES':1, 'NO ':0})
dataset_no_null['CylinderStat'] = dataset_no_null['CylinderStat'].map({'YES':1, 'NO ':0})

#viewing the modified data
print(dataset_no_null.info(verbose=True, null_counts=True))

In [ ]:
#viewing a summary description of the remaining categorical variables in the modified data
dataset_no_null.describe(include = ['object']).T

,count,unique,top,freq
EnergyRating,195312,15,D1,26275
VentilationMethod,195312,6,Natural vent.,192809
SuspendedWoodenFloor,195312,3,No,177214
CombiBoiler,195312,4,None,188944
PrimaryCircuitLoss,195312,9,Boiler with uninsulated primar,127963
ThermalMassCategory,195312,5,Medium,110215


In [ ]:
#Encoding the categorical variables that have more than 2 unique values
#Using One Hot Encoding
encoded_data = get_dummies(dataset_no_null, columns = ['VentilationMethod','SuspendedWoodenFloor','CombiBoiler','PrimaryCircuitLoss','ThermalMassCategory'], drop_first=True) #dropping the first columns for feature selection

#viewing the modified data
print(encoded_data.info(verbose=True, null_counts=True))

In [ ]:
#Viewing the shape of the encoded data
encoded_data.shape

(195312, 70)

In [ ]:
#writing a method to create a new target column BER_Efficiency, where an energy rating of A or B will have a value of 'High' and any others will have a value of 'Low'.
def categorize_ber(rating):
    high_efficiency = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3']
    return 'High' if rating in high_efficiency else 'Low'

# Applying the function to create the new column
encoded_data['BER_Efficiency'] = encoded_data['EnergyRating'].apply(categorize_ber)

# Displaying the first few rows to verify the new column is added correctly
encoded_data.head()

,EnergyRating,GroundFloorArea(sq m),UValueWall,UValueRoof,UValueFloor,UValueWindow,UvalueDoor,WallArea,RoofArea,FloorArea,WindowArea,DoorArea,NoStoreys,CO2Rating,HSMainSystemEfficiency,MultiDwellingMPRN,HSSupplSystemEff,WHMainSystemEff,SupplSHFuel,SupplWHFuel,NoOfChimneys,NoOfOpenFlues,NoOfFansAndVents,NoOfFluelessGasFires,DraftLobby,FanPowerManuDeclaredValue,HeatExchangerEff,PercentageDraughtStripped,NoOfSidesSheltered,PermeabilityTestResult,TempAdjustment,HeatSystemControlCat,HeatSystemResponseCat,NoCentralHeatingPumps,CHBoilerThermostatControlled,NoOilBoilerHeatingPumps,OBBoilerThermostatControlled,OBPumpInsideDwelling,NoGasBoilerHeatingPumps,WarmAirHeatingSystem,DistributionLosses,StorageLosses,SolarHotWaterHeating,ElecImmersionInSummer,CylinderStat,LowEnergyLightingPercent,LivingAreaPercent,RoomInRoofArea,VentilationMethod_Bal.whole mech.vent no heat re,VentilationMethod_Natural vent.,VentilationMethod_Pos input vent.- loft,VentilationMethod_Pos input vent.- outside,VentilationMethod_Whole house extract vent.,SuspendedWoodenFloor_Yes (Sealed),SuspendedWoodenFloor_Yes (Unsealed),"CombiBoiler_Instantaneous, without keep-ho",CombiBoiler_None,CombiBoiler_Storage combi boiler store vol,PrimaryCircuitLoss_Boiler with insulated primary,PrimaryCircuitLoss_Boiler with uninsulated primar,PrimaryCircuitLoss_CPSU (including electric CPSU),PrimaryCircuitLoss_Combi boiler,PrimaryCircuitLoss_Community heating,PrimaryCircuitLoss_Electric immersion heater,PrimaryCircuitLoss_None,PrimaryCircuitLoss_Separate boiler and thermal st,ThermalMassCategory_Low,ThermalMassCategory_Medium,ThermalMassCategory_Medium-high,ThermalMassCategory_Medium-low,BER_Efficiency
0,C2,100.00,0.55,0.29,0.44,3.10,3.00,76.81,104.30,67.50,18.00,3.70,2,46.05,95.0,0,55.0,95.0,15.0,1.0,0,1,0,0.0,0,0.0,0.0,100,2,0.0,0.0,2,1,1,0,1,0,0,0,0,333.686,1,0,0,0,90,21.58000,0.00,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,Low
1,D1,103.30,1.09,0.26,0.47,2.60,3.00,70.63,108.19,108.19,14.58,3.82,1,62.91,85.3,0,65.0,85.3,15.0,1.0,1,0,1,0.0,0,0.0,0.0,100,1,0.0,0.6,1,1,1,0,1,0,0,0,0,0.000,1,0,0,0,0,22.99130,0.00,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,Low
2,D1,152.13,0.28,1.64,0.73,2.30,2.88,102.76,92.91,63.83,24.48,3.41,2,59.55,85.0,0,55.0,85.0,15.0,28.0,3,0,1,0.0,0,0.0,0.0,100,2,0.0,0.6,1,1,1,0,1,0,0,0,0,437.274,1,0,0,0,50,9.94544,29.93,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,Low
3,C2,157.63,0.29,0.18,0.34,2.61,3.00,109.15,84.12,92.28,23.84,1.85,2,46.51,84.2,0,30.0,84.2,15.0,28.0,1,0,5,0.0,0,0.0,0.0,100,3,0.0,0.6,1,1,1,0,1,0,0,0,0,447.242,1,0,1,0,10,10.79740,0.00,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,Low
4,G,69.30,1.78,0.13,0.75,2.80,3.10,73.87,69.30,69.30,12.89,3.93,1,98.06,78.1,0,30.0,78.1,15.0,1.0,2,1,7,0.0,0,0.0,0.0,90,3,0.0,0.6,1,1,1,0,0,0,0,1,0,265.070,1,0,0,0,22,23.10250,0.00,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,Low


In [ ]:
#dropping the EnergyRating column since we now have a new target column
encoded_data = encoded_data.drop('EnergyRating', axis = 1)

#Encoding the BER_Efficiency column using Mapping
encoded_data['BER_Efficiency'] = encoded_data['BER_Efficiency'].map({'High':1, 'Low':0})

#viewing the modified data
print(encoded_data.info(verbose=True, null_counts=True))

#encoded_data['BER_Efficiency'].value_counts()

In [ ]:
#Encoding the BER_Efficiency column using Mapping
encoded_data['BER_Efficiency'] = encoded_data['BER_Efficiency'].map({'High':1, 'Low':0})

#viewing the modified data
print(encoded_data.info(verbose=True, null_counts=True))

In [ ]:
# Dividing dataset into label and feature sets
X = encoded_data.drop(['BER_Efficiency'], axis = 1) # Features
Y = encoded_data['BER_Efficiency'] # Label
print(X.shape)
print(Y.shape)

(195312, 69)
(195312,)


In [ ]:
# Normalizing numerical features so that each feature has mean 0 and variance 1
X_scaled = StandardScaler().fit_transform(X)

In [ ]:
#Splitting the data into training and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size = 0.2, random_state = 100)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(156249, 69)
(39063, 69)
(156249,)
(39063,)


In [ ]:
#Balancing the data using oversampling technique

#viewing the distribution of observations before oversampling
print("Number of observations in each class before oversampling (training data): \n", Series(Y_train).value_counts())

X_train,Y_train = SMOTE(random_state=100).fit_resample(X_train, Y_train) #oversampling

#viewing the distribution of observations after oversampling
print("Number of observations in each class after oversampling (training data): \n", Series(Y_train).value_counts())

Number of observations in each class before oversampling (training data): 
 0    136097
1     20152
Name: BER_Efficiency, dtype: int64
Number of observations in each class after oversampling (training data): 
 0    136097
1    136097
Name: BER_Efficiency, dtype: int64


# **Implementing Decision Tree**

In [ ]:
#Decision Tree Classifier
DT_classifier = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 15)# classifier building
DT_classifier.fit(X_train, Y_train) # training
Y_pred = DT_classifier.predict(X_test) # testing
imp_features_dt = Series(DT_classifier.feature_importances_, index=list(X)).sort_values(ascending=False) # what are the important features?
print('IMPORTANT FEATURES')
print('------------------------- ')
print(imp_features_dt) #printing the important features

IMPORTANT FEATURES
------------------------- 
CO2Rating                                              0.849913
NoGasBoilerHeatingPumps                                0.066008
SupplSHFuel                                            0.029547
HSSupplSystemEff                                       0.007569
HSMainSystemEfficiency                                 0.006379
UValueFloor                                            0.004663
NoOilBoilerHeatingPumps                                0.004493
UValueWindow                                           0.002433
UValueWall                                             0.002425
LowEnergyLightingPercent                               0.002410
WHMainSystemEff                                        0.002399
FloorArea                                              0.002136
GroundFloorArea(sq m)                                  0.001730
UValueRoof                                             0.001710
WallArea                                               0.0

In [ ]:
# MODEL EVALUATION
con_matrix = metrics.confusion_matrix(Y_test, Y_pred) #calculating confusion matrix
print("Confusion Matrix: ")
print (con_matrix)

accuracy = metrics.accuracy_score(Y_test, Y_pred) #calculating accuracy
print("Accuracy: ", accuracy)

# using 'F1' for scoring because we are placing equal emphasis on minimizing both false positives and false negatives)
f1 = metrics.f1_score(Y_test, Y_pred) #calculating f1
print("F1: ", f1)

Accuracy:  0.9833346133169495
[[33545   445]
 [  206  4867]]
f1 : 0.9373134328358209


# **Implementing AdaBoost**

In [ ]:
# Implementing AdaBoost
# Tuning the AdaBoost parameter 'n_estimators' and implementing cross-validation using Grid Search
model_ab = Pipeline([('balancing', SMOTE(random_state = 101)),('classification', AdaBoostClassifier(random_state = 1))])
grid_param = {'classification__n_estimators': [2, 5, 10, 20, 50]}

#using 'F1' for scoring because we are placing equal emphasis on minimizing both false positives and false negatives)
gd_sr = GridSearchCV(estimator = model_ab, param_grid = grid_param, scoring = 'f1', cv = 5)

In [ ]:
#training, testing, evaluation and ranking
gd_sr.fit(X_scaled, Y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('balancing', SMOTE(random_state=101)),
                                       ('classification',
                                        AdaBoostClassifier(random_state=1))]),
             param_grid={'classification__n_estimators': [2, 5, 10, 20, 50]},
             scoring='f1')

In [ ]:
#getting the best no of n_estimators
best_parameters = gd_sr.best_params_
print('The best no of n_estimators = ',best_parameters) #printing

#getting the best f1 score based on the grid search
best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print('The best f1 score = ', best_result) #printing

#getting the important features list for the best model
imp_features_ab = pd.Series(gd_sr.best_estimator_.named_steps["classification"].feature_importances_, index=list(X)).sort_values(ascending=False)
print('IMPORTANT FEATURES')
print('------------------------- ')
print(imp_features_ab) #printing the important features

The best no of n_estimators =  {'classification__n_estimators': 50}
The best f1 score =  0.9223894375047734
IMPORTANT FEATURES
------------------------- 
CO2Rating                                              0.28
SupplSHFuel                                            0.14
GroundFloorArea(sq m)                                  0.06
WHMainSystemEff                                        0.06
UValueFloor                                            0.06
UValueWindow                                           0.06
UValueWall                                             0.06
HeatSystemControlCat                                   0.06
LowEnergyLightingPercent                               0.02
CylinderStat                                           0.02
OBBoilerThermostatControlled                           0.02
NoOfSidesSheltered                                     0.02
NoOfChimneys                                           0.02
NoOilBoilerHeatingPumps                                0.02
UValue

# **Implementing Random Forest**

In [ ]:
# Implementing Random Forest Classifier
# Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
model_rf = Pipeline([('balancing', SMOTE(random_state = 101)),('classification', RandomForestClassifier(criterion='entropy', max_features='auto', random_state=1))])
no_of_trees = {'classification__n_estimators': [250, 300, 350, 400, 450]}

#using 'F1' for scoring because we are placing equal emphasis on minimizing both false positives and false negatives)
grid_search = GridSearchCV(estimator = model_rf, param_grid = no_of_trees, scoring='f1', cv=5)

In [ ]:
#training, testing, evaluation and ranking
grid_search.fit(X_scaled, Y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('balancing', SMOTE(random_state=101)),
                                       ('classification',
                                        RandomForestClassifier(criterion='entropy',
                                                               max_features='auto',
                                                               random_state=1))]),
             param_grid={'classification__n_estimators': [250, 300, 350, 400,
                                                          450]},
             scoring='f1')

In [ ]:
#getting the best no of n_estimators
best_parameters = grid_search.best_params_
print('The best no of n_estimators = ',best_parameters) #printing

#getting the best f1 score based on the grid search
best_result = grid_search.best_score_ # Mean cross-validated score of the best_estimator
print('The best f1 score = ', best_result) #printing

#getting the important features list
imp_features_rf = pd.Series(grid_search.best_estimator_.named_steps["classification"].feature_importances_, index=list(X)).sort_values(ascending=False)
print('IMPORTANT FEATURES')
print('------------------------- ')
print(imp_features_rf) #printing the important features

The best no of n_estimators =  {'classification__n_estimators': 450}
The best f1 score =  0.9446254655442337
IMPORTANT FEATURES
------------------------- 
CO2Rating                                              0.330263
UValueFloor                                            0.073988
UValueWindow                                           0.064265
TempAdjustment                                         0.062768
UValueWall                                             0.054584
CHBoilerThermostatControlled                           0.048300
HeatSystemControlCat                                   0.045474
WHMainSystemEff                                        0.031556
HSMainSystemEfficiency                                 0.031530
UValueRoof                                             0.029474
ElecImmersionInSummer                                  0.015736
GroundFloorArea(sq m)                                  0.013354
NoCentralHeatingPumps                                  0.013328
DistributionL